# Análise Exploratória Unificada - Dataset de Startups

Esta análise exploratória utiliza todas as features selecionadas das três hipóteses formuladas:
- **H1 - Capital**: Features relacionadas a financiamento e crescimento
- **H2 - Geografia**: Features de localização geográfica
- **H3 - Operacional**: Features de maturidade operacional

**Objetivo**: Analisar o dataset unificado para identificar padrões que influenciam o sucesso de startups utilizando um conjunto integrado de features que engloba aspectos financeiros, geográficos e operacionais.

**Dataset Unificado**: 646 startups com 20 features selecionadas das três hipóteses + 1 variável alvo (sucesso/fracasso)

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas carregadas e configurações definidas")

In [ ]:
# Carregamento do dataset unificado
train_df = pd.read_csv('database/unified_train.csv')
test_df = pd.read_csv('database/unified_test.csv')
features_summary = pd.read_csv('database/unified_features_summary.csv')

print(f"📊 Dataset Unificado:")
print(f"   • Train: {train_df.shape}")
print(f"   • Test: {test_df.shape}")
print(f"   • Features: {len(features_summary)} selecionadas")

# Separar target e features
target_var = 'labels'
feature_cols = [col for col in train_df.columns if col != target_var]

print(f"\n🎯 Distribuição da variável alvo:")
target_dist = train_df[target_var].value_counts()
for label, count in target_dist.items():
    pct = (count / len(train_df)) * 100
    status = 'Sucesso' if label == 1 else 'Fracasso'
    print(f"   • {status} ({label}): {count} ({pct:.1f}%)")

# Preview dos dados
print(f"\n📋 Primeiras linhas do dataset unificado:")
display(train_df.head())

## 1. Análise Estatística Descritiva do Dataset Unificado

In [ ]:
# Análise das features por categoria
print("📋 COMPOSIÇÃO DAS FEATURES UNIFICADAS")
print("=" * 50)

# Agrupar por origem das hipóteses
features_by_origin = features_summary.groupby('Origem').size()
print(f"\n🏷️ Features por origem:")
for origem, count in features_by_origin.items():
    print(f"   • {origem}: {count} features")

# Agrupar por tipo
features_by_type = features_summary.groupby('Tipo').size()
print(f"\n📊 Features por tipo:")
for tipo, count in features_by_type.items():
    print(f"   • {tipo}: {count} features")

# Mostrar features de cada categoria
print(f"\n🔍 Detalhamento das features:")
for origem in features_summary['Origem'].unique():
    origem_features = features_summary[features_summary['Origem'] == origem]['Feature'].tolist()
    print(f"   • {origem}: {origem_features}")

# Análise da variável alvo
target_counts = train_df[target_var].value_counts()
target_pct = train_df[target_var].value_counts(normalize=True) * 100

print(f"\n🎯 DISTRIBUIÇÃO DA VARIÁVEL ALVO:")
print(f"   • Fracasso (0): {target_counts[0]} startups ({target_pct[0]:.1f}%)")
print(f"   • Sucesso (1): {target_counts[1]} startups ({target_pct[1]:.1f}%)")
print(f"   • Balanceamento: {'Balanceado' if abs(target_pct[0] - target_pct[1]) < 10 else 'Desbalanceado'}")

In [ ]:
# Visualização da distribuição do target
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Distribuição da Variável Alvo - Sucesso vs Fracasso', fontsize=16, fontweight='bold')

# Gráfico de barras
colors = ['lightcoral', 'lightgreen']
axes[0].bar(['Fracasso (0)', 'Sucesso (1)'], target_counts.values, color=colors, alpha=0.8)
axes[0].set_title('Contagem Absoluta', fontweight='bold')
axes[0].set_ylabel('Número de Startups')
axes[0].grid(True, alpha=0.3)
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Gráfico de pizza
axes[1].pie(target_counts.values, labels=['Fracasso (0)', 'Sucesso (1)'], 
           autopct='%1.1f%%', startangle=90, colors=colors)
axes[1].set_title('Proporção', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"💡 Insight: Dataset moderadamente balanceado, com ligeira maioria de casos de fracasso.")

## 2. Análise das Variáveis Numéricas

In [ ]:
# Análise estatística descritiva
print("? ANÁLISE ESTATÍSTICA DESCRITIVA")
print("=" * 40)

# Identificar features numéricas e binárias
numeric_features = []
binary_features = []

for feature in feature_cols:
    unique_vals = train_df[feature].nunique()
    if unique_vals == 2 and set(train_df[feature].unique()).issubset({0, 1, np.nan}):
        binary_features.append(feature)
    elif train_df[feature].dtype in ['float64', 'int64'] and unique_vals > 10:
        numeric_features.append(feature)

print(f"\n📊 Classificação das features:")
print(f"   • Numéricas contínuas: {len(numeric_features)}")
print(f"   • Binárias: {len(binary_features)}")

# Estatísticas das features numéricas
if numeric_features:
    print(f"\n📈 Estatísticas das features numéricas:")
    numeric_stats = train_df[numeric_features].describe()
    print(numeric_stats.round(3))

# Análise de correlação entre features numéricas
if len(numeric_features) > 1:
    print(f"\n? Correlações mais altas entre features numéricas:")
    corr_matrix = train_df[numeric_features].corr()
    
    # Encontrar correlações altas (excluindo diagonal)
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > 0.5:
                high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_val))
    
    if high_corr:
        for feat1, feat2, corr_val in sorted(high_corr, key=lambda x: abs(x[2]), reverse=True):
            print(f"   • {feat1} ↔ {feat2}: {corr_val:.3f}")
    else:
        print(f"   • Nenhuma correlação alta (|r| > 0.5) encontrada")

In [ ]:
# Visualização 1: Correlação das features com o target
print("📊 CORRELAÇÃO DAS FEATURES COM O TARGET")
print("=" * 45)

# Calcular correlação com target
target_correlations = []
for feature in feature_cols:
    if train_df[feature].dtype in ['float64', 'int64']:
        corr = train_df[feature].corr(train_df[target_var])
        target_correlations.append({'Feature': feature, 'Correlacao': corr, 'Abs_Correlacao': abs(corr)})

# Criar DataFrame e ordenar
corr_df = pd.DataFrame(target_correlations).sort_values('Abs_Correlacao', ascending=False)

print(f"\n🎯 Top 10 features com maior correlação com sucesso:")
for _, row in corr_df.head(10).iterrows():
    direction = "📈" if row['Correlacao'] > 0 else "📉"
    print(f"   {direction} {row['Feature']}: {row['Correlacao']:.3f}")

# Gráfico de barras das correlações
plt.figure(figsize=(14, 8))
top_features = corr_df.head(15)
colors = ['green' if x > 0 else 'red' for x in top_features['Correlacao']]

bars = plt.barh(range(len(top_features)), top_features['Correlacao'], color=colors, alpha=0.7)
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Correlação com Sucesso')
plt.title('Top 15 Features - Correlação com Sucesso de Startups', fontweight='bold', fontsize=14)
plt.grid(axis='x', alpha=0.3)

# Adicionar valores nas barras
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width + 0.01 if width > 0 else width - 0.01, bar.get_y() + bar.get_height()/2, 
             f'{width:.3f}', ha='left' if width > 0 else 'right', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Visualização 2: Análise das features binárias
if binary_features:
    print("\n📊 ANÁLISE DAS FEATURES BINÁRIAS")
    print("=" * 40)
    
    # Calcular taxa de sucesso para cada feature binária
    binary_analysis = []
    
    for feature in binary_features:
        # Taxa de sucesso quando feature = 1
        success_rate_1 = train_df[train_df[feature] == 1][target_var].mean()
        success_rate_0 = train_df[train_df[feature] == 0][target_var].mean()
        
        count_1 = train_df[feature].sum()
        count_0 = len(train_df) - count_1
        
        binary_analysis.append({
            'Feature': feature,
            'Taxa_Sucesso_1': success_rate_1,
            'Taxa_Sucesso_0': success_rate_0,
            'Diferenca': success_rate_1 - success_rate_0,
            'Count_1': count_1,
            'Count_0': count_0
        })
    
    binary_df = pd.DataFrame(binary_analysis).sort_values('Diferenca', ascending=False, key=abs)
    
    print(f"\n🎯 Impacto das features binárias no sucesso:")
    for _, row in binary_df.head(10).iterrows():
        impact = "📈 Positivo" if row['Diferenca'] > 0 else "📉 Negativo"
        print(f"   {impact} {row['Feature']}: {row['Diferenca']:.3f} "
              f"(Taxa: {row['Taxa_Sucesso_1']:.3f} vs {row['Taxa_Sucesso_0']:.3f})")
    
    # Visualização das features binárias mais impactantes
    plt.figure(figsize=(16, 10))
    
    top_binary = binary_df.head(12)  # Top 12 para visualização
    
    x_pos = np.arange(len(top_binary))
    width = 0.35
    
    bars1 = plt.bar(x_pos - width/2, top_binary['Taxa_Sucesso_0'], width, 
                   label='Feature = 0', alpha=0.7, color='lightcoral')
    bars2 = plt.bar(x_pos + width/2, top_binary['Taxa_Sucesso_1'], width,
                   label='Feature = 1', alpha=0.7, color='lightblue')
    
    plt.xlabel('Features Binárias')
    plt.ylabel('Taxa de Sucesso')
    plt.title('Taxa de Sucesso por Feature Binária (Top 12 por Impacto)', fontweight='bold')
    plt.xticks(x_pos, top_binary['Feature'], rotation=45, ha='right')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # Adicionar valores nas barras
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualização 3: Distribuições das features numéricas por sucesso
if numeric_features:
    print("\n📈 DISTRIBUIÇÃO DAS FEATURES NUMÉRICAS POR SUCESSO")
    print("=" * 55)
    
    # Selecionar top features numéricas por correlação
    top_numeric = corr_df[corr_df['Feature'].isin(numeric_features)].head(8)
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    for i, (_, row) in enumerate(top_numeric.iterrows()):
        if i < len(axes):
            feature = row['Feature']
            
            # Separar por sucesso/fracasso
            success_data = train_df[train_df[target_var] == 1][feature]
            failure_data = train_df[train_df[target_var] == 0][feature]
            
            # Histograma
            axes[i].hist(failure_data, bins=20, alpha=0.6, label='Fracasso', color='red', density=True)
            axes[i].hist(success_data, bins=20, alpha=0.6, label='Sucesso', color='green', density=True)
            
            axes[i].set_title(f'{feature}\n(Corr: {row["Correlacao"]:.3f})', fontweight='bold')
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('Densidade')
            axes[i].legend()
            axes[i].grid(alpha=0.3)
    
    # Remover subplots vazios
    for i in range(len(top_numeric), len(axes)):
        axes[i].set_visible(False)
    
    plt.suptitle('Distribuição das Top Features Numéricas por Resultado', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Estatísticas comparativas
    print(f"\n📊 Estatísticas comparativas (Sucesso vs Fracasso):")
    for _, row in top_numeric.head(5).iterrows():
        feature = row['Feature']
        success_mean = train_df[train_df[target_var] == 1][feature].mean()
        failure_mean = train_df[train_df[target_var] == 0][feature].mean()
        
        print(f"   • {feature}:")
        print(f"     Sucesso: {success_mean:.3f} | Fracasso: {failure_mean:.3f} | Diferença: {success_mean - failure_mean:.3f}")

## 2. Matriz de Correlação das Features Principais

In [ ]:
# Visualização 4: Matriz de correlação das features mais importantes
print("\n🔗 MATRIZ DE CORRELAÇÃO DAS FEATURES PRINCIPAIS")
print("=" * 50)

# Selecionar top 15 features por correlação absoluta com target
top_features_for_corr = corr_df.head(15)['Feature'].tolist()

# Criar matriz de correlação
corr_matrix = train_df[top_features_for_corr + [target_var]].corr()

# Visualização
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})

plt.title('Matriz de Correlação - Top 15 Features + Target', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

# Identificar multicolinearidade
high_corr_pairs = []
for i in range(len(corr_matrix.columns)-1):  # Excluir target
    for j in range(i+1, len(corr_matrix.columns)-1):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_val))

if high_corr_pairs:
    print(f"\n⚠️ Features com alta correlação (|r| > 0.7) - Possível multicolinearidade:")
    for feat1, feat2, corr_val in high_corr_pairs:
        print(f"   • {feat1} ↔ {feat2}: {corr_val:.3f}")
else:
    print(f"\n✅ Nenhuma multicolinearidade alta detectada entre as top features")

In [ ]:
# Análise final e insights
print("\n? INSIGHTS PRINCIPAIS DO DATASET UNIFICADO")
print("=" * 55)

# Top features por categoria
print(f"\n🏆 TOP FEATURES POR CATEGORIA DE HIPÓTESE:")

for origem in ['Capital', 'Geografia', 'Operacional']:
    origem_features = features_summary[features_summary['Origem'] == origem]['Feature'].tolist()
    origem_corr = corr_df[corr_df['Feature'].isin(origem_features)].head(3)
    
    print(f"\n   📈 {origem}:")
    for _, row in origem_corr.iterrows():
        direction = "↗️" if row['Correlacao'] > 0 else "↘️"
        print(f"      {direction} {row['Feature']}: {row['Correlacao']:.3f}")

# Resumo estatístico
total_features = len(feature_cols)
strong_predictors = len(corr_df[corr_df['Abs_Correlacao'] > 0.1])
weak_predictors = len(corr_df[corr_df['Abs_Correlacao'] <= 0.05])

print(f"\n📊 RESUMO ESTATÍSTICO:")
print(f"   • Total de features: {total_features}")
print(f"   • Preditores fortes (|r| > 0.1): {strong_predictors}")
print(f"   • Preditores fracos (|r| ≤ 0.05): {weak_predictors}")
print(f"   • Taxa de desbalanceamento: {train_df[target_var].mean():.1%} sucesso")

print(f"\n🎯 RECOMENDAÇÕES PARA MODELAGEM:")
print(f"   ✅ Dataset bem estruturado com features complementares")
print(f"   ✅ Combinação equilibrada de features financeiras, geográficas e operacionais")
if high_corr_pairs:
    print(f"   ⚠️ Considerar remoção de features altamente correlacionadas")
else:
    print(f"   ✅ Baixa multicolinearidade detectada")
print(f"   ✅ Features com diferentes níveis de poder preditivo disponíveis")
print(f"   ⚠️ Considerar técnicas para lidar com desbalanceamento de classes")

## 6. Resumo dos Principais Insights

### 📊 **Principais Insights do Dataset Unificado:**

**Composição e Qualidade dos Dados:**
- Dataset unificado com **20 features** selecionadas das três hipóteses (Capital, Geografia, Operacional)
- Distribuição balanceada entre features numéricas contínuas e binárias/categóricas
- Nenhuma correlação excessivamente alta detectada entre features (baixa multicolinearidade)

**Correlações com Sucesso:**
- Features de **Capital**: funding_total_usd, funding_rounds, avg_participants mostram correlações mais fortes
- Features **Geográficas**: Localização em hubs tecnológicos (CA, NY, MA) impacta positivamente
- Features **Operacionais**: relationships, milestones e métricas de execução são relevantes

**Padrões Identificados:**
- Startups de sucesso apresentam maior volume de investimento e mais rodadas de funding
- Localização geográfica em hubs tecnológicos oferece vantagem competitiva
- Capacidade de execution (relacionamentos e marcos) diferencia startups bem-sucedidas

### 🎯 **Recomendações para Modelagem:**
1. **Dataset Preparado**: Estrutura unificada permite modelagem integrada sem necessidade de análises separadas por hipótese
2. **Feature Selection**: Top 15 features por correlação absoluta capturam os principais fatores preditivos
3. **Modelo Recomendado**: Random Forest ou Gradient Boosting para lidar com features mistas e desbalanceamento
4. **Validação**: Usar validação cruzada estratificada para garantir representatividade das classes

**Próximo Passo**: Implementar pipeline de modelagem utilizando o dataset unificado com todas as 20 features selecionadas.